In [4]:
import warnings
warnings.filterwarnings("ignore")

import pickle
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.layers.recurrent import LSTM
import numpy as np

# A simple text generator using LSTM layers


In [5]:
def LoadText():
    print('reading data')
    # with open('../fox_scrapes/deadpool2_online_articles.txt', "r") as text_file:
    with open('/Users/nikolaushbuenning/ebooks/great_expectations', "r") as text_file:
        data = text_file.read().lower()
    data = data.replace('--nextarticle', '')
    text = list(data)
    inputSize = len(text)
    data = sorted(list(set(text)))
    dataSize = len(data)
    return text, inputSize, dataSize, data

## Clean the data and one hot encode the characters

In [6]:
np.random.seed(1)
text, textSize, uniqueChars, chars = LoadText()
print('done reading data')
ci = dict((c,i) for i, c in enumerate(chars))
ic = dict((i,c) for i, c in enumerate(chars))

seqLen = 80
sentences = []
nextChars = []
print('looping through text, creating sentence')
for i in range(0, textSize-seqLen, 3):
    sentences.append(text[i:i+seqLen])
    nextChars.append(text[i+seqLen])

x_train = np.zeros((len(sentences), seqLen, uniqueChars), dtype=np.bool)
y_train = np.zeros((len(sentences), uniqueChars), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for j, char in enumerate(sentence):
        x_train[i, j, ci[char]] = 1
    y_train[i, ci[nextChars[i]]] = 1



reading data
done reading data
looping through text, creating sentence


## Build and compile model

In [7]:
print('building model')
model = Sequential()
model.add(LSTM(150, input_shape=(seqLen, uniqueChars), return_sequences=True, implementation=1))
model.add(Dropout(0.15))
model.add(LSTM(150, input_shape=(seqLen, uniqueChars), implementation=1))
model.add(Dropout(0.15))
model.add(Dense(uniqueChars, activation='softmax'))
model.compile(optimizer=keras.optimizers.rmsprop(lr=0.007), loss='categorical_crossentropy')

building model


##  Train and output after each 20 epochs

In [ ]:
n_epochs = 20
n_iters = 5

print('iterating')
file_obj = open('generated.txt', 'w')
for it in range(n_iters):
    print('=' * 50)
    print('iteration: ', it)
    model.fit(x_train, y_train, batch_size=128, epochs=n_epochs, verbose=1)
    startIndex = np.random.randint(0, textSize-seqLen-1)
    generated = ''
    sentence = text[startIndex:startIndex+seqLen]
    generated = generated.join(sentence)
    print("generating with seed: ", generated)
    if it > -1:
        for i in range(4000):
            x = np.zeros((1, seqLen, uniqueChars))
            for j, char in enumerate(sentence):
                x[0, j, ci[char]] = 1

            prediction = model.predict(x, batch_size=1, verbose=0)[0]
            index = np.random.choice(uniqueChars, p=prediction)
            # print([index, prediction])
            char = ic[index]
            generated += char
            sentence.append(char)
            sentence.pop(0)
        file_obj.write('--------- next generated text')
        file_obj.write(generated)
    print('generated text: ')
    print(generated)
file_obj.close()
# model.save('predator_model0.h5')

## Or one training iteration

In [9]:
model.fit(x_train, y_train, batch_size=128, epochs=20, verbose=1)

Epoch 1/20
331526/331526 [==============================] - 1344s 4ms/step - loss: 1.8217
Epoch 2/20
331526/331526 [==============================] - 1883s 6ms/step - loss: 1.5836
Epoch 3/20
331526/331526 [==============================] - 1439s 4ms/step - loss: 1.5161
Epoch 4/20
331526/331526 [==============================] - 1385s 4ms/step - loss: 1.4823
Epoch 5/20
331526/331526 [==============================] - 1368s 4ms/step - loss: 1.4568
Epoch 6/20
331526/331526 [==============================] - 1239s 4ms/step - loss: 1.4405
Epoch 7/20
331526/331526 [==============================] - 1282s 4ms/step - loss: 1.4260
Epoch 8/20
331526/331526 [==============================] - 1346s 4ms/step - loss: 1.4134
Epoch 9/20
331526/331526 [==============================] - 1368s 4ms/step - loss: 1.4047
Epoch 10/20
331526/331526 [==============================] - 1368s 4ms/step - loss: 1.3993
Epoch 11/20
331526/331526 [==============================] - 1287s 4ms/step - loss: 1.3978
Epoch 12

In [19]:
startIndex = np.random.randint(0, textSize-seqLen-1)
generated = ''
sentence = text[startIndex:startIndex+seqLen]
generated = generated.join(sentence)
for i in range(4000):
    x = np.zeros((1, seqLen, uniqueChars))
    for j, char in enumerate(sentence):
        x[0, j, ci[char]] = 1

    prediction = model.predict(x, batch_size=1, verbose=0)[0]
    index = np.random.choice(uniqueChars, p=prediction)
    # print([index, prediction])
    char = ic[index]
    generated += char
    sentence.append(char)
    sentence.pop(0)

In [20]:
print(generated)

l, and had been for
some time silently meeting mr. jaggers's look. when i did at the knobling even a fattentions i lose the intentions at that of mlountital night, what i understand it
a sided worstance to cutleth that he is, i am
very oster either as!”

i had not fele. which there were hands as to suppose having been it, but for her eges. he had no would try into my sister's. “of long hurt?” he and we being at the christy to the water return at have in the day, only complated down from the house joe and not concarsed to make us, i replied with a coysfaded on a gall dumprocked restlemen. i one occursed surpiigh to all his
now, which full as exhent medily afoched of the same knowledges, the palect of me us aa no, you have been knew?” sit inso bridenbod towards no little garding and but
i opened me sat an agke
of fire to-fortute very
garden persons and rich-morry about the some moment, or
became every hit, in somewed him. provinfness. would pretty brought it
the eluvin boby, master, and 

##### Ok, so not perfect. But, it kind of works; may need some hyperparameter tuning.